In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d omkargurav/face-mask-dataset

In [ ]:
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

In [ ]:
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
with_mask_files = os.listdir('/content/data/with_mask')

In [ ]:
without_mask_files=os.listdir('/content/data/without_mask')

In [ ]:
len(with_mask_files)

In [ ]:
len(without_mask_files)

In [ ]:
with_mask_label=[1]*3725
without_mask_label=[0]*3828

In [ ]:
labels=with_mask_label+without_mask_label

In [ ]:
img=mpimg.imread('/content/data/with_mask/with_mask_1.jpg')
imgplot=plt.imshow(img)

In [ ]:
with_mask_path='/content/data/with_mask/'

In [ ]:
data=[]

In [ ]:
for img in with_mask_files:
  image=Image.open(with_mask_path+img)
  image=image.resize((128,128))
  image=image.convert('RGB')
  image=np.array(image)
  data.append(image)

In [ ]:
without_mask_path='/content/data/without_mask/'

In [ ]:
for img in without_mask_files:
  image=Image.open(without_mask_path+img)
  image=image.resize((128,128))
  image=image.convert('RGB')
  image=np.array(image)
  data.append(image)

In [ ]:
X=np.array(data)
Y=np.array(labels)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [ ]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes=2
model=keras.Sequential()

In [ ]:
model.add(keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(keras.layers.Flatten())

In [ ]:
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.5))

In [ ]:
model.add(keras.layers.Dense(num_of_classes,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
hist=model.fit(X_train_scaled,Y_train,validation_split=0.1,epochs=5)

In [ ]:
input_image_path=input("Enter the path of the image")
input_image=cv2.imread(input_image_path)
cv2_imshow(input_image)

In [ ]:
input_image_resize=cv2.resize(input_image,(128,128))
input_image_scaled=input_image_resize/255
input_image_reshaped=np.reshape(input_image_scaled,[1,128,128,3])
prediction=model.predict(input_image_reshaped)

In [ ]:
final=np.argmax(prediction)

In [ ]:
if final==1:
  print("Ther person is wearing a mask")
else:
  print("The person is not wearing a mask")